In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
import pandas as pd
data_taka= pd.read_csv('/content/drive/MyDrive/dataset/instagram_taka_posts.csv',encoding='utf-8')
data_taka.head()

,url,caption,hashtags,likes,image_url
0,https://www.instagram.com/p/DJtN4Nkzgrq/,⠀\n디토는 지금 일본 다카마쓰 여행 중🇯🇵\n작은 도시에서 큰 행복 찾고 싶다면?...,NaN,115.0,https://scontent-icn2-1.cdninstagram.com/v/t51...
1,https://www.instagram.com/p/DJBf6jnJ6R5/,💎 일본 다카마쓰 - 우동투어\n강나미 따라 간 당일치기 우동투어🍜\n\n🎀 하시모...,"#うどん, #우동, #udong, #일본우동, #강나미우동투어, #일본우동, #일본...",0.0,https://scontent-icn2-1.cdninstagram.com/v/t51...
2,https://www.instagram.com/p/C--G8cASZUj/,"일본 다카마쓰에서 꼭 들려야 할 '붓쇼잔 온천'\n\n노천탕과 실내 온탕, 냉탕으로...",NaN,1.0,https://scontent-icn2-1.cdninstagram.com/v/t51...
3,https://www.instagram.com/p/By311fWguYn/,토토로❤️ #일본지브리샵 #지브리샵 #다카마쓰지브리 #다카마쓰상가 #다카마츠 #다카...,"#일본지브리샵, #지브리샵, #다카마쓰지브리, #다카마쓰상가, #다카마츠, #다카마...",22.0,https://scontent-icn2-1.cdninstagram.com/v/t51...
4,https://www.instagram.com/p/DLUokhUhW30/,야돈우동붱\n\n#다카마쓰 #다카마쓰여행 #일본 #일본여행 #香川屋本店,"#다카마쓰, #다카마쓰여행, #일본, #일본여행, #香川屋本店",0.0,https://scontent-icn2-1.cdninstagram.com/v/t51...


In [111]:
from collections import Counter
import re

all_words = ' '.join(data_taka['caption'].astype(str)).split()

cleaned_words = [
    word for word in all_words
    if len(word) > 1 or word.startswith('#')
    and not re.fullmatch(r'[\W_]+', word)
]


word_counts = Counter(cleaned_words)


most_common_words = word_counts.most_common(10)

# 출력
for word, count in most_common_words:
  print(f'{word}: {count}')
  pass
print(f'\n총 고유 단어 수: {len(word_counts)}')

일본: 117
다카마쓰: 115
우동: 103
#다카마쓰: 100
있는: 78
여행: 73
#다카마츠: 73
#다카마쓰여행: 73
#일본여행: 65
#일본: 41

총 고유 단어 수: 10341


In [112]:
tourist_spots = [
    '나오시마', '리쓰린', '시코쿠',
    '고토히라', '쇼도시마', '쇼도지마', '올리브', '사누키',
    '제면소', '우동투어', '구라시키', '쿠사마', '야마고에우동',
    '츠루마루', '메리켄야', '오카야마', '붓쇼잔', '온천',
    '가마타마', '바카이치다이', '다카마쓰항', '시코쿠무라',
    '야쿠리지', '키쿠게츠테이', '효고마치', '심볼타워', '타마모',
    '돗토리', '곤피라','호박','야요이'
]
transactions=[]

for caption in data_taka['caption']:
    places = [spot for spot in tourist_spots if spot in caption]

    hashtags = re.findall(r'#\w+', caption)

    # tourist_spots 단어가 해시태그에 포함된 경우만 허용
    filtered_hashtags = [tag for tag in hashtags if any(spot in tag for spot in tourist_spots)]

    transaction = places + filtered_hashtags
    transactions.append(transaction)

group_map = {
    '온천':'붓쇼잔',
    '제면소':'우동투어',
 '쿠사마': '나오시마',
    '야요이':'나오시마',
    '호박':'나오시마',
    '야마고에우동':'우동투어',
    '츠루마루':'우동투어',
    '메리켄야':'우동투어',
    '바카이치다이':'우동투어',
    '곤피라':'우동투어',
    '사누키':'우동투어',
    '올리브':'쇼도시마',
    '시코쿠':'시코쿠무라',

    # 추가 그룹핑 가능
}

#'숨길 거':'나타낼 거'

transactions = []

for caption in data_taka['caption']:
    found_spots = [spot for spot in tourist_spots if spot in caption]
    grouped_places = [group_map.get(spot, spot) for spot in found_spots]
    # 중복 제거
    unique_places = list(dict.fromkeys(grouped_places))

    hashtags = re.findall(r'#\w+', caption)
    filtered_mapped_hashtags = []
    for tag in hashtags:
        tag_text = tag[1:]
        matched_spots = [spot for spot in tourist_spots if spot in tag_text]
        if matched_spots:
            mapped = group_map.get(matched_spots[0], matched_spots[0])
            filtered_mapped_hashtags.append(mapped)
    # 중복 제거
    unique_hashtags = list(dict.fromkeys(filtered_mapped_hashtags))

    # places 와 hashtags 합치고, 전체 중복 다시 제거 (필요하면)
    transaction = unique_places + unique_hashtags
    transaction = list(dict.fromkeys(transaction))  # 전체 중복 제거

    transactions.append(transaction)

for i, t in enumerate(transactions[:30]):
    print(f"{i+1}번째 transaction:", t)



1번째 transaction: []
2번째 transaction: ['우동투어']
3번째 transaction: ['붓쇼잔']
4번째 transaction: []
5번째 transaction: []
6번째 transaction: []
7번째 transaction: ['리쓰린', '고토히라']
8번째 transaction: ['우동투어', '붓쇼잔']
9번째 transaction: ['우동투어', '가마타마']
10번째 transaction: ['시코쿠무라', '우동투어']
11번째 transaction: []
12번째 transaction: ['리쓰린']
13번째 transaction: []
14번째 transaction: ['쇼도시마']
15번째 transaction: ['구라시키']
16번째 transaction: []
17번째 transaction: ['리쓰린', '쇼도시마', '우동투어', '붓쇼잔', '다카마쓰항']
18번째 transaction: []
19번째 transaction: ['나오시마']
20번째 transaction: []
21번째 transaction: []
22번째 transaction: ['리쓰린', '붓쇼잔']
23번째 transaction: ['리쓰린', '우동투어', '붓쇼잔', '돗토리']
24번째 transaction: []
25번째 transaction: ['시코쿠무라']
26번째 transaction: []
27번째 transaction: []
28번째 transaction: []
29번째 transaction: ['리쓰린', '키쿠게츠테이']
30번째 transaction: []


In [119]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

# ✔️ 1. 빈 트랜잭션 제거
cleaned_transactions = [t for t in transactions if t]

# ✔️ 2. 인코딩
te = TransactionEncoder()
te_ary = te.fit(cleaned_transactions).transform(cleaned_transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# ✔️ 3. 자주 등장하는 아이템셋
frequent_itemsets = apriori(df, min_support=0.04, use_colnames=True)
frequent_itemsets_sorted = frequent_itemsets.sort_values(by='support', ascending=False)
# ✔️ 4. 연관 규칙 추출
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)

print('자주 등장하는 아이템셋')
print(frequent_itemsets_sorted[frequent_itemsets_sorted['support'] >= 0.1])
print("\n🔗 연관 규칙 (Top 10 by Lift):")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by='lift', ascending=False).head(100))

자주 등장하는 아이템셋
    support itemsets
9  0.304054   (우동투어)
2  0.236486   (나오시마)
7  0.202703  (시코쿠무라)
4  0.189189    (리쓰린)
5  0.141892    (붓쇼잔)
6  0.135135   (쇼도시마)
0  0.121622   (고토히라)

🔗 연관 규칙 (Top 10 by Lift):
  antecedents consequents   support  confidence      lift
0      (고토히라)       (붓쇼잔)  0.040541    0.333333  2.349206
1       (붓쇼잔)      (고토히라)  0.040541    0.285714  2.349206
2       (리쓰린)       (붓쇼잔)  0.047297    0.250000  1.761905
3       (붓쇼잔)       (리쓰린)  0.047297    0.333333  1.761905
4      (우동투어)       (붓쇼잔)  0.054054    0.177778  1.252910
5       (붓쇼잔)      (우동투어)  0.054054    0.380952  1.252910
6      (우동투어)      (쇼도시마)  0.047297    0.155556  1.151111
7      (쇼도시마)      (우동투어)  0.047297    0.350000  1.151111
8      (우동투어)     (시코쿠무라)  0.060811    0.200000  0.986667
9     (시코쿠무라)      (우동투어)  0.060811    0.300000  0.986667


In [114]:
# 고토히라
# 붓쇼잔 온천
# 리쓰린공원
# 시코쿠무라
# 쇼도시마

# (우동투어)